In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## Problem 9.1 Data clustering

In [4]:
data = np.loadtxt('data.txt', delimiter=',')

## Problem 9.2 Principal component analysis

In [10]:
traj = np.loadtxt('traj.xyz')

ValueError: Wrong number of columns at line 2

In [94]:
def load_traj(file):
    with open(file,'r') as f:
        file = [line.rstrip('\n') for line in f]
        traj = []

        for i in range(0, len(file), 9):
            coordinates = file[i+2: i+9]
            traj.append(coordinates)
        traj = np.array(traj)

        dtraj = []
        for i in range(len(traj)):
            block = []
            for k in range(len(traj[i])):
                row = traj[i][k].split()
                row = row[1:]
                block.append(row)
            dtraj.append(block)
        dtraj = np.array(dtraj, dtype='np.float32')

    return dtraj

In [95]:
traj = load_traj('traj.xyz')

TypeError: data type "np.float32" not understood

In [92]:
print(traj)

[[['29.650002' '16.470001' '27.490002']
  ['28.410000' '15.780001' '27.100000']
  ['28.320000' '15.390000' '25.630001']
  ...
  ['27.310001' '14.620000' '25.200003']
  ['27.140003' '14.140000' '23.850002']
  ['26.180000' '15.010000' '23.059999']]

 [['28.990002' '16.150002' '23.750000']
  ['27.720001' '16.570000' '23.160002']
  ['27.820000' '16.670000' '21.650002']
  ...
  ['26.700001' '16.650002' '20.930002']
  ['26.470001' '16.690001' '19.500000']
  ['25.250000' '17.560001' '19.260002']]

 [['27.770002' '18.460001' '27.500002']
  ['26.770000' '19.260002' '26.790001']
  ['27.000000' '19.160002' '25.290001']
  ...
  ['25.860003' '19.040001' '24.600000']
  ['25.910000' '19.120001' '23.150002']
  ['25.520000' '20.510002' '22.680000']]

 ...

 [['23.000002' '29.270000' '28.870001']
  ['21.880001' '28.350000' '28.600002']
  ['22.480001' '26.950001' '28.580000']
  ...
  ['21.690002' '25.870001' '28.549999']
  ['22.150002' '24.530001' '28.830002']
  ['21.680002' '23.600002' '27.720001']]

 [